Search Problem

In [3]:
from simpleai.search import SearchProblem, astar, greedy

GOAL = '1-2-3\n8-e-4\n7-6-5'
INITIAL = '8-1-2\n7-e-3\n4-5-6'

def list_to_string(list_):
    return '\n'.join(['-'.join(row) for row in list_])

def string_to_list(string_):
    return [row.split('-') for row in string_.split('\n')]

def find_location(rows, element_to_find):
    for ir, row in enumerate(rows):
        for ic, element in enumerate(row):
            if element == element_to_find:
                return ir, ic

goal_positions = {}
rows_goal = string_to_list(GOAL)
for number in '12345678e':
    goal_positions[number] = find_location(rows_goal, number)

class EightPuzzleProblem(SearchProblem):
    def actions(self, state):
        rows = string_to_list(state)
        row_e, col_e = find_location(rows, 'e')
        actions = []
        if row_e > 0:
            actions.append(rows[row_e - 1][col_e])
        if row_e < 2:
            actions.append(rows[row_e + 1][col_e])
        if col_e > 0:
            actions.append(rows[row_e][col_e - 1])
        if col_e < 2:
            actions.append(rows[row_e][col_e + 1])
        return actions

    def result(self, state, action):
        rows = string_to_list(state)
        row_e, col_e = find_location(rows, 'e')
        row_n, col_n = find_location(rows, action)
        rows[row_e][col_e], rows[row_n][col_n] = rows[row_n][col_n], rows[row_e][col_e]
        return list_to_string(rows)

    def is_goal(self, state):
        return state == GOAL

    def cost(self, state1, action, state2):
        return 1

    def heuristic(self, state):
        rows = string_to_list(state)
        distance = 0
        for number in '12345678e':
            row_n, col_n = find_location(rows, number)
            row_n_goal, col_n_goal = goal_positions[number]
            distance += abs(row_n - row_n_goal) + abs(col_n - col_n_goal)
        return distance


In [ ]:
result = greedy(EightPuzzleProblem(INITIAL))
for action, state in result.path():
    print('Move number', action)
    print(state)


In [ ]:
result = astar(EightPuzzleProblem(INITIAL))
for action, state in result.path():
    print('Move number', action)
    print(state)


Local Search

In [9]:
import math
import random

GOAL = [
    [1, 2, 3],
    [8, 0, 4],
    [7, 6, 5],
]


# Nếu phép lai crosover tạo ra các số trùng lặp thì điều chỉnh lại kết quả để đảm báo đúng với yêu cầu (8 số là 1-8 và ô trống là số 0)
def AdjustState(state):
    state1d = Convert2DTo1DList(state)
    stateset = set(state1d)
    missingset = set([0, 1, 2, 3, 4, 5, 6, 7, 8])
    missingset.difference_update(stateset)
    remain = list(missingset)
    for i in range(len(state1d) - 1):
        for j in range(i + 1, len(state1d)):
            if state1d[i] == state1d[j]:
                state1d[j] = remain[0]
                remain.remove(remain[0])
                break
    return Convert1DTo2DList(state1d)


# Chuyển List 1 chiều sang list hai chiều
def Convert1DTo2DList(input_list):
    list2d = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    for i in range(3):
        for j in range(3):
            list2d[i][j] = input_list[i * 3 + j]
    return list2d


# Chuyển list 2 chiều sang list 1 chiều
def Convert2DTo1DList(input_list):
    list1d = []
    for i in range(3):
        for j in range(3):
            list1d.append(input_list[i][j])
    return list1d

# Tìm vị trí của ô trống (số 0)
def FindZero(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return [i, j]
    return None

# Hoán đổi vị trí 2 ô
def Swap(state, i, j, ii, jj):
    statecopy = state.copy()
    temp = statecopy[i][j]
    statecopy[i][j] = statecopy[ii][jj]
    statecopy[ii][jj] = temp
    return statecopy

# Chuyển list 2 chiều về dạng xâu để in ra đáp số bằng hàm print
def Convert2DListToString(input_list):
    return "\n".join(["".join(str(x)) for x in input_list]).replace(",", " ").replace("0", " ")


##################################
# Để sử dụng các thuật toán local search cần một hàm value để xác định giá trị của các thuật toán local search
# Với thuật toán Genetic Algorithm, cần yêu cầu thêm 3 hàm generate_random_state (tạo cấu hình ban đầu), crossover (lai tạo), mutate (tạo đột biến)
# Với thuật toán Hill Climbing, cần yêu cầu thêm 3 hàm actions, result và cost. Ngoài ra, còn cần initial_state để khởi tạo cấu hình ban đầu
# Với thuật toán Simulated Annealing, cần yêu cần thêm 
class EightNumbersProblem:
    # Giá trị sẽ được xác định dựa trên số lượng các ô nằm đúng vị trí.
    # Càng nhiều ô nằm đúng vị trí thì càng tốt - đó là mục tiêu của các thuật toán local search
    def value(self, state):
        sum = 0
        for i in range(3):
            for j in range(3):
                if state[i][j] == GOAL[i][j]:
                    sum += 1
        return sum

    initial_state = [
        [1, 2, 3],
        [7, 6, 4],
        [5, 8, 0],
    ]

    # Hàm action cho các thuật toán local search khác GA
    def actions(self, state):
        actions = []
        [i, j] = FindZero(state)
        if i>0:
            actions.append([i-1, j])
        if i<2:
            actions.append([i+1, j])
        if j>0:
            actions.append([i, j-1])
        if j<2:
            actions.append([i, j+1])
        random.shuffle(actions)
        return actions

    #Thực thi result
    def result(self, state, action):
        new_state = state.copy()
        [i, j] = FindZero(state)nh
        Swap(new_state, action[0], action[1], i, j)
        return new_state

    #Tính cost cho mỗi lần dịch chuyển
    def cost(self, state, action, state2):
        return 1

    # Sinh cấu hình ban đầu một cách ngẫu nhiên
    # Tạo một list 1 chiều gồm 8 số và ô trống (số 0). Cho nhảy ngẫu nhiêu thứ tự bằng shuffle, sau đó chuyển list 1 chiều sang list 2 chiều
    def generate_random_state(self):
        orin = [1, 2, 3, 4, 5, 6, 7, 8, 0]
        random.shuffle(orin)
        init_state = Convert1DTo2DList(orin)
        return init_state


    # Lai tạo 2 cấu hình: cấu hình 1 chỉ lấy một dòng, cấu hình 2 lấy 2 dòng
    # Sau đó ghép lại để tạo ra list 2 chiều union
    # Tuy nhiên, trong union sẽ có các số trùng lặp, nên sử dụng AdjustState để chuyển đổi về dạng thức hợp lệ
    # Ví dụ, nếu union có dạng [[1 2 3], [3 5 6], [7 8 0]] (lưu ý số 3 lặp 2 lần và thiếu số 4)
    # Khi đó, số lặp lần thứ 2 sẽ lấy giá trị là số còn thiếu (nếu nhiều số lặp lại thì lấy theo thứ tự các số còn thiếu) => Xem chi tiết trong AdjustState
    def crossover(self, state1, state2):
        cut_first = state1[0][:]
        cut_second = state2[1:3][:]
        union = [cut_first, cut_second[0], cut_second[1]]
        union = AdjustState(union)
        return union

    # Phép tạo đột biến: sẽ sinh ngẫu nhiên một cặp chỉ số i, j và hoán đổi giá trị của 2 phần tử (i, j) và (j, i)
    # Để đảm bảo trong trường hợp nếu i=j vẫn có sự thay đổi cấu hình, thì khi i=j, ta sẽ tăng chỉ số i lên 1 (nếu vượt quá 2 thì sẽ quay về 0)
    def mutate(self, state):
        i, j = random.randint(0, 2), random.randint(0, 2)
        mutated = state.copy()
        if i == j:
            i = (i + 1) % 3
        temp = mutated[i][j]
        mutated[i][j] = mutated[j][i]
        mutated[j][i] = temp
        return mutated

    # Hàm biểu diễn trạng thái kết quả
    def state_representation(self, state):
        return Convert2DListToString(state)

In [7]:
goal_str = '''[1  2  3]
              [8     4]
              [7  6  5]'''

In [ ]:
from simpleai.search.local import hill_climbing

result2 = hill_climbing(problem)
for i in range(100000):
    result2 = hill_climbing(problem)
    if result2.state_representation() == goal_str:
        break
print(result2.state_representation())

In [12]:
def simulated_annealing(problem, initial_state, max_steps, initial_temp, cooling_rate):
    current_state = initial_state
    current_value = problem.value(current_state)
    temperature = initial_temp

    for step in range(max_steps):
        if temperature <= 0:
            break

        next_state = problem.result(current_state, random.choice(problem.actions(current_state)))
        next_value = problem.value(next_state)
        delta_value = next_value - current_value

        if delta_value > 0 or math.exp(delta_value / temperature) > random.random():
            current_state = next_state
            current_value = next_value

        temperature *= cooling_rate

    return current_state

In [ ]:
problem = EightNumbersProblem()
initial_state = problem.generate_random_state()
solved_state = simulated_annealing(problem, initial_state, max_steps=10000, initial_temp=100, cooling_rate=0.99)

print("Initial State:")
print(Convert2DListToString(initial_state))
print("\nSolved State:")
print(Convert2DListToString(solved_state))

In [ ]:
from simpleai.search.local import genetic

problem = EightNumbersProblem()
result = genetic(problem, population_size=100, iterations_limit=50, mutation_chance=0.1)
print(result.state_representation())

Constraint Satisfaction Problem

In [6]:
from simpleai.search import CspProblem, backtrack

# Định nghĩa trạng thái đích
GOAL_STATE = (1, 2, 3, 8, 0, 4, 7, 6, 5)  # 0 đại diện cho ô trống

def create_domain():
    return list(range(9))

def create_constraints():
    constraints = []
    
    # Các số phải khác nhau
    for i in range(9):
        for j in range(i + 1, 9):
            constraints.append(((i, j), lambda x, y: x != y))
    
    # Các số phải ở đúng vị trí theo trạng thái đích
    for i in range(9):
        constraints.append(((i,), lambda x, goal=GOAL_STATE[i]: x == goal))
    
    return constraints

def solve_eight_puzzle(initial_state):
    variables = list(range(9))
    domains = {v: create_domain() for v in variables}
    
    # Giới hạn miền giá trị cho trạng thái ban đầu
    for i, value in enumerate(initial_state):
        if value != 0:
            domains[i] = [value]
    
    constraints = create_constraints()
    
    problem = CspProblem(variables, domains, constraints)
    
    result = backtrack(problem)
    
    if result:
        return tuple(result[var] for var in variables)
    else:
        return None

def print_puzzle_state(state):
    for i in range(3):
        for j in range(3):
            value = state[i*3 + j]
            if value == 0:
                print(" ", end=" ")
            else:
                print(value, end=" ")
        print()  # Xuống dòng sau mỗi hàng

# Ví dụ sử dụng
initial_state = (1, 2, 3, 8, 4, 0, 7, 6, 5)
print("Trạng thái ban đầu:")
print_puzzle_state(initial_state)

solution = solve_eight_puzzle(initial_state)

if solution:
    print("\nGiải pháp tìm được:")
    print_puzzle_state(solution)
else:
    print("\nKhông tìm thấy giải pháp.")

Trạng thái ban đầu:
1 2 3 
8 4   
7 6 5 

Không tìm thấy giải pháp.


In [11]:
from simpleai.search import CspProblem, backtrack

# Định nghĩa trạng thái đích
GOAL_STATE = (1, 2, 3, 4, 5, 6, 7, 8, 0)  # 0 đại diện cho ô trống

def create_domain():
    return list(range(9))

def create_constraints():
    constraints = []

    # Các số phải khác nhau
    for i in range(9):
        for j in range(i + 1, 9):
            constraints.append(((i, j), lambda x, y: x != y))
    
    return constraints

def solve_eight_puzzle(initial_state):
    variables = list(range(9))
    domains = {v: create_domain() for v in variables}
    
    # Giới hạn miền giá trị cho trạng thái ban đầu
    for i, value in enumerate(initial_state):
        if value != 0:
            domains[i] = [value]
    
    constraints = create_constraints()
    
    problem = CspProblem(variables, domains, constraints)
    
    result = backtrack(problem)
    
    if result:
        return tuple(result[var] for var in variables)
    else:
        return None

def print_puzzle_state(state):
    for i in range(3):
        for j in range(3):
            value = state[i*3 + j]
            if value == 0:
                print(" ", end=" ")
            else:
                print(value, end=" ")
        print()  # Xuống dòng sau mỗi hàng

# Ví dụ sử dụng
initial_state = (1, 2, 3, 8, 4, 7, 0, 5, 6)
print("Trạng thái ban đầu:")
print_puzzle_state(initial_state)

solution = solve_eight_puzzle(initial_state)

if solution:
    print("\nGiải pháp tìm được:")
    print_puzzle_state(solution)
else:
    print("\nKhông tìm thấy giải pháp.")


Trạng thái ban đầu:
1 2 3 
8 4 7 
  5 6 

Giải pháp tìm được:
1 2 3 
8 4 7 
  5 6 


In [15]:
from simpleai.search import CspProblem, backtrack

# Định nghĩa trạng thái đích
GOAL_STATE = (1, 2, 3, 8, 0, 4, 7, 6, 5)

# Định nghĩa hàm kiểm tra ràng buộc
def constraint_adjacent(variables, values):
    pos1, pos2 = variables
    val1, val2 = values
    if val1 == 0 or val2 == 0:  # Ô trống có thể di chuyển
        return abs(pos1 - pos2) == 1 or abs(pos1 - pos2) == 3
    return True  # Các ô khác không di chuyển

def constraint_goal_state(variables, values):
    return values[0] == GOAL_STATE[variables[0]]

# Định nghĩa bài toán
variables = list(range(9))
domains = {}
for var in variables:
    domains[var] = list(range(9))  # Mỗi ô có thể chứa giá trị từ 0 đến 8

constraints = [
    ((0, 1), constraint_adjacent),
    ((0, 3), constraint_adjacent),
    ((1, 2), constraint_adjacent),
    ((1, 4), constraint_adjacent),
    ((2, 5), constraint_adjacent),
    ((3, 4), constraint_adjacent),
    ((3, 6), constraint_adjacent),
    ((4, 5), constraint_adjacent),
    ((4, 7), constraint_adjacent),
    ((5, 8), constraint_adjacent),
    ((6, 7), constraint_adjacent),
    ((7, 8), constraint_adjacent),
]

# Thêm ràng buộc trạng thái đích
for i in range(9):
    constraints.append(((i,), constraint_goal_state))

problem = CspProblem(variables, domains, constraints)

# Giải bài toán
result = backtrack(problem)

# In kết quả
if result:
    print("Giải pháp tìm được:")
    for i in range(3):
        for j in range(3):
            print(result[i*3 + j], end=" ")
        print()
else:
    print("Không tìm thấy giải pháp.")

Giải pháp tìm được:
1 2 3 
8 0 4 
7 6 5 
